In [185]:
from selenium import webdriver
import time
import pandas
import csv
from datetime import datetime

In [186]:
details_mapping = {
    "COLOUR": "EXTERIOR COLOUR",
    "VARIANT": "BADGE",
    "KMS SHOWING": "ODOMETER",
    "MOTOR TYPE": "ENGINE",
    "GEARBOX MAKE/MODEL": "TRANSMISSION",
    "STEERING TYPE": "STEERING",
    "ENGINE": "ENGINE NUMBER",
    "MOTOR TYPE": "ENGINE"}

In [187]:
RESEARCH_BASE_URL = "https://www.slatteryauctions.com.au/search/cars-commercial-vehicles-motorcycles?gclid=Cj0KCQjwl8XtBRDAARIsAKfwtxBX33_4QMER8CNoUeURBhw1Q4IStUCf7ORD8CnKeSoWvAW2wa5U3ToaApoqEALw_wcB"
EXECUTABLE_PATH = "/home/saronida/lib/geckodriver"

In [188]:
driver = webdriver.Firefox(executable_path=EXECUTABLE_PATH)

In [189]:
driver.get(RESEARCH_BASE_URL)

In [190]:
time.sleep(10)
item_cards = driver.find_elements_by_class_name("item-card__inner")

In [191]:
len(item_cards)

74

In [192]:
cars_urls = [item_card.find_element_by_tag_name("a").get_attribute("href") for item_card in item_cards]

In [193]:
first_example_url = "https://www.slatteryauctions.com.au/product/4459/950529/2017-lamborghini-aventador-lp-750-4-car-auction/2017-lamborghini-aventador"

In [194]:
first_example_url

'https://www.slatteryauctions.com.au/product/4459/950529/2017-lamborghini-aventador-lp-750-4-car-auction/2017-lamborghini-aventador'

In [195]:
driver.get(first_example_url)

In [196]:
link = first_example_url

In [197]:
title = driver.find_elements_by_class_name("product-details__title")[1].text

In [198]:
title

'2017 LAMBORGHINI AVENTADOR'

In [199]:
price = driver.find_element_by_class_name("starting-bid").find_element_by_class_name("ng-binding").text

In [200]:
price

'$623,000.00'

In [201]:
initial_details = {
    "TITLE": title, "LINK": link, "TIMESTAMP": int(datetime.timestamp(datetime.now()))}

In [202]:
initial_details["DEALER NAME"] = "Slattery Auctions"

In [203]:
location = driver.find_element_by_class_name("snapshot__details").find_elements_by_tag_name("li")[2].text

In [204]:
location

'6-8 Waterview Close, Dandenong South VIC 3175'

In [205]:
initial_details["LOCATION"] = location

In [206]:
initial_details

{'TITLE': '2017 LAMBORGHINI AVENTADOR',
 'LINK': 'https://www.slatteryauctions.com.au/product/4459/950529/2017-lamborghini-aventador-lp-750-4-car-auction/2017-lamborghini-aventador',
 'TIMESTAMP': 1576704909,
 'DEALER NAME': 'Slattery Auctions',
 'LOCATION': '6-8 Waterview Close, Dandenong South VIC 3175'}

In [207]:
##############

In [208]:
product_details_comments_features = driver.find_element_by_id("product-details")

In [209]:
description = product_details_comments_features.find_elements_by_class_name("table-product-details")[0]

In [210]:
description_items = description.text.split("\n")

In [211]:
description_details = [item for item in description_items if ":" in item and len(item.split(":"))==2]

In [212]:
details_block = product_details_comments_features.find_elements_by_class_name("table-product-details")[1]

In [213]:
details = details_block.find_elements_by_tag_name("tr")

In [214]:
len(details)

12

In [215]:
asset_conditions_block = product_details_comments_features.find_elements_by_class_name("table-product-details")[2]

In [216]:
asset_conditions = asset_conditions_block.find_elements_by_tag_name("tr")

In [217]:
len(asset_conditions)

10

In [218]:
extra_block = product_details_comments_features.find_elements_by_class_name("table-product-details")[3]

In [219]:
extra = extra_block.find_elements_by_tag_name("tr")

In [220]:
len(extra)

14

In [221]:
#"text-success" in extra[0].find_element_by_tag_name("td").find_element_by_tag_name("i").get_attribute("class")

In [222]:
#main_pane = driver.find_element_by_id("product-details-main-pane")

In [223]:
#main_pane_details_block = main_pane.find_element_by_class_name("table-product-details")

In [224]:
#main_pane_details = main_pane_details_block.find_elements_by_tag_name("tr")

In [225]:
#extra[0].find_element_by_tag_name("td").find_element_by_tag_name("i").get_attribute("class")

In [226]:
def parse_details(description_details, details, asset_conditions, extra, initial_details):
    """ Parses vehicle's details box. """

    parsed_details = initial_details
    
    for detail in description_details:
        key = detail.split(":")[0]
        value = detail.split(":")[1]
        if value != "":
            parsed_details[key] = value
            
    for detail in details:
        parsed_details[detail.find_element_by_tag_name("th").text] = detail.find_element_by_tag_name("td").text
    
    for detail in asset_conditions:
        parsed_details[detail.find_element_by_tag_name("th").text] = detail.find_element_by_tag_name("td").text
    
    features_list = []
    for feature in extra:
        if "text-success" in feature.find_element_by_tag_name("td").find_element_by_tag_name("i").get_attribute("class"):
            features_list.append(feature.find_element_by_tag_name("th").text)
    parsed_details["VEHICLE FEATURES"] = ",".join(features_list)
    
    parsed_details_df = pandas.DataFrame.from_dict(parsed_details, orient="index", columns=["value"])
    parsed_details_df["key"] = parsed_details_df.index
    parsed_details_df.reset_index(inplace=True, drop=True)
    return parsed_details_df

In [227]:
parsed_details_df = parse_details(description_details, details, asset_conditions, extra, initial_details)

In [228]:
parsed_details_df

,value,key
0,2017 LAMBORGHINI AVENTADOR,TITLE
1,https://www.slatteryauctions.com.au/product/44...,LINK
2,1576704909,TIMESTAMP
3,Slattery Auctions,DEALER NAME
4,"6-8 Waterview Close, Dandenong South VIC 3175",LOCATION
5,ZHWET3ZD9HLA06405,VIN
6,L53906796,ENGINE
7,GREY,COLOUR
8,7037,ODOMETER
9,01/06/2017,COMPLIANCE DATE


In [229]:
def alter_details(parsed_details_df):
    """ Alters details to match mapping format. """

    parsed_details_df = parsed_details_df[~pandas.isnull(parsed_details_df.key)]
    parsed_details_df["key"] = parsed_details_df["key"].apply(lambda key: key.replace(":", "").strip().upper())
    parsed_details_df["key"] = parsed_details_df["key"].apply(
        lambda key: details_mapping[key] if key in details_mapping.keys() else key)
    parsed_details_df.drop_duplicates(subset ="key", inplace = True)
    return parsed_details_df

In [230]:
parsed_details_df = alter_details(parsed_details_df)

In [231]:
tmp_dict = parsed_details_df.to_dict(orient="list")
parsed_details = dict(zip(tmp_dict["key"], tmp_dict["value"]))

In [232]:
parsed_details["VEHICLE FEATURES"]

'Key,A/C,ABS,Power Windows,Drivable,Central Locking,Cruise Control,Power Mirrors,Airbag(s),Alloys,CD Player'

In [233]:
parsed_details_df

,value,key
0,2017 LAMBORGHINI AVENTADOR,TITLE
1,https://www.slatteryauctions.com.au/product/44...,LINK
2,1576704909,TIMESTAMP
3,Slattery Auctions,DEALER NAME
4,"6-8 Waterview Close, Dandenong South VIC 3175",LOCATION
5,ZHWET3ZD9HLA06405,VIN
6,L53906796,ENGINE NUMBER
7,GREY,EXTERIOR COLOUR
8,7037,ODOMETER
9,01/06/2017,COMPLIANCE DATE


In [235]:
with open('../results/slattery.csv', 'w') as f:
    w = csv.DictWriter(f, parsed_details.keys())
    w.writeheader()
    w.writerow(parsed_details)
